# Manutenzione preventiva - 

# Manutenzione preventiva - DoS Attacks

In [3]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np

In [78]:
clean_cap_30m = open('.\\captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json')
data_cc_30m = json.load(clean_cap_30m)
clean_cap_30m.close()

In [88]:
interesting_layers = ["eth", "arp", "icmp", "ip", "tcp", "udp", "mbtcp"]
timestamp_field_name = 'frame.time_epoch'
ipat_field_name = 'inter.packet_arrival_time'
eth_fields = ['eth.dst', 'eth.src', 'eth.type', 'eth.len']
arp_fields = ['arp.hw.type', 'arp.proto.type', 'arp.hw.size', 'arp.proto.size']
icmp_fields = ['icmp.type', 'icmp.code', 'icmp.checksum', 'icmp.checksum.status']
ip_fields = ['ip.version', 'ip.hdr_len', 'ip.dsfield','ip.len', 'ip.id', 'ip.flags', 'ip.ttl', 'ip.proto', 'ip.checksum', 'ip.checksum.status', 'ip.src', 'ip.dst']
tcp_fields = ['tcp.srcport', 'tcp.dstport', 'tcp.stream', 'tcp.len', 'tcp.seq', 'tcp.nxtseq', 'tcp.ack',
              'tcp.hdr_len', 'tcp.flags', 'tcp.window_size_value', 'tcp.window_size', 'tcp.window_size_scalefactor', 'tcp.checksum',
              'tcp.checksum.status', 'tcp.urgent_pointer']
udp_fields = ['udp.srcport', 'udp.dstport', 'udp.port', 'udp.length', 'udp.checksum', 'udp.checksum_status']
mbtcp_fields = ['mbtcp.trans_id', 'mbtcp.prot_id', 'mbtcp.len', 'mbtcp.unit_id']
interesting_layer_fields = [eth_fields, arp_fields, icmp_fields, ip_fields, tcp_fields, udp_fields, mbtcp_fields]
df_columns = [timestamp_field_name] + eth_fields + arp_fields + icmp_fields + ip_fields + tcp_fields + udp_fields + mbtcp_fields
packets_values = []

In [90]:
counter = 0
for packet in data_cc_30m:
    tmp_pkt = []

    tmp_pkt.append(packet['_source']['layers']['frame']['frame.time_epoch'])
    
    for i in range(len(interesting_layers)):
        if interesting_layers[i] in packet['_source']['layers']:
            for field in interesting_layer_fields[i]:
                if field in packet['_source']['layers'][interesting_layers[i]]:
                    tmp_pkt.append(packet['_source']['layers'][interesting_layers[i]][field])
                else:
                    tmp_pkt.append(np.nan)
        else:
            for field in interesting_layer_fields[i]:
                tmp_pkt.append(np.nan)
    packets_values.append(tmp_pkt)

print(len(packets_values[1]))
print(len(packets_values))
      

50
144300


In [91]:
# Costruzione dataframe
df_clean_30m = pd.DataFrame(data=packets_values, columns=df_columns)
# Conversione colonna timestamp a float
df_clean_30m['frame.time_epoch'] = df_clean_30m['frame.time_epoch'].astype('float')
# Inserimento colonna inter packet arrival time
df_clean_30m[ipat_field_name] = df_clean_30m['frame.time_epoch'].diff()

df_clean_30m.columns

Index(['frame.time_epoch', 'eth.dst', 'eth.src', 'eth.type', 'eth.len',
       'arp.hw.type', 'arp.proto.type', 'arp.hw.size', 'arp.proto.size',
       'icmp.type', 'icmp.code', 'icmp.checksum', 'icmp.checksum.status',
       'ip.version', 'ip.hdr_len', 'ip.dsfield', 'ip.len', 'ip.id', 'ip.flags',
       'ip.ttl', 'ip.proto', 'ip.checksum', 'ip.checksum.status', 'ip.src',
       'ip.dst', 'tcp.srcport', 'tcp.dstport', 'tcp.stream', 'tcp.len',
       'tcp.seq', 'tcp.nxtseq', 'tcp.ack', 'tcp.hdr_len', 'tcp.flags',
       'tcp.window_size_value', 'tcp.window_size',
       'tcp.window_size_scalefactor', 'tcp.checksum', 'tcp.checksum.status',
       'tcp.urgent_pointer', 'udp.srcport', 'udp.dstport', 'udp.port',
       'udp.length', 'udp.checksum', 'udp.checksum_status', 'mbtcp.trans_id',
       'mbtcp.prot_id', 'mbtcp.len', 'mbtcp.unit_id',
       'inter.packet_arrival_time'],
      dtype='object')

In [92]:
'''df1=df_clean_30m.filter(like='udp', axis=1)
print(len(df1))

df_clean = df1.dropna(subset=['udp.srcport'])

print(df_clean['udp.srcport'])
print(len(df_clean['udp.srcport']))'''



144300
8         59926
95           68
178       59926
183          68
462          68
          ...  
143551    59926
143759       68
143977       68
144123       68
144293    59926
Name: udp.srcport, Length: 1522, dtype: object
1522


In [93]:
# Conversione del MAC Address da esadecimale a decimale
# flags_ tutti nulli//
# ip.checksum_status tutti nulli//
# arp tutti null//
# icmp tutti nulli (da verificare)//
# tcp.checksum_status tutti null
# eliminare le colonne che non servono piu alla fine e inserire i campi binari e vedere il numero di feature
df_clean_30m['eth.dst_cleaned'] = df_clean_30m['eth.dst'].str.replace(":", "", regex=False)
df_clean_30m['eth.dst_int'] = df_clean_30m['eth.dst_cleaned'].apply(lambda x : int(x, 16))
df_clean_30m['eth.src_cleaned'] = df_clean_30m['eth.src'].str.replace(":", "", regex=False)
df_clean_30m['eth.src_int'] = df_clean_30m['eth.src_cleaned'].apply(lambda x : int(x, 16))
df_clean_30m['eth.type'] = df_clean_30m['eth.type'].apply(
    lambda x: int(x, 16) if pd.notnull(x) and isinstance(x, str) else np.nan
)
df_clean_30m['eth.len'] = df_clean_30m['eth.len'].astype(float)
df_clean_30m.drop(['eth.dst_cleaned', 'eth.dst', 'eth.src_cleaned', 'eth.src'], axis=1)

df_clean_30m['ip.version'] = df_clean_30m['ip.version'].astype(float)
df_clean_30m['ip.hdr_len'] = df_clean_30m['ip.hdr_len'].astype(float)
df_clean_30m['ip.src'].replace({np.nan: '169.254.0.0'}, inplace=True)
df_clean_30m['ip.dst'].replace({np.nan: '169.254.0.0'}, inplace=True)
df_clean_30m['ip.src'] = df_clean_30m['ip.src'].apply(lambda x : int(ip.IPv4Address(x)))
df_clean_30m['ip.dst'] = df_clean_30m['ip.dst'].apply(lambda x : int(ip.IPv4Address(x)))
df_clean_30m['ip.dsfield'] = df_clean_30m['ip.dsfield'].apply(
    lambda x: int(x, 16) if pd.notnull(x) and isinstance(x, str) else np.nan
)
'''df_clean_30m['ip.dsfield_dscp'] = df_clean_30m['ip.dsfield_dscp'].astype(float)
df_clean_30m['ip.dsfield_ecn'] = df_clean_30m['ip.dsfield_ecn'].astype(float)'''
df_clean_30m['ip.len'] = pd.to_numeric(df_clean_30m['ip.len'], errors='coerce').astype('Int64')
df_clean_30m['ip.id'] = df_clean_30m['ip.id'].apply(
    lambda x: int(x, 16) if pd.notnull(x) and isinstance(x, str) and len(x.strip()) > 0 else np.nan
)

df_clean_30m['ip.flags'] = df_clean_30m['ip.flags'].apply(
    lambda x: int(x, 16) if pd.notnull(x) and isinstance(x, str) and len(x.strip()) > 0 else np.nan
)
'''df_clean_30m['ip.frag_offset'] = df_clean_30m['ip.frag_offset'].astype(float)'''
df_clean_30m['ip.ttl'].fillna(0, inplace=True)

# Convertire in intero
df_clean_30m['ip.ttl'] = df_clean_30m['ip.ttl'].astype(int)
df_clean_30m['ip.proto'].fillna(0, inplace=True)

# Ora puoi convertire la colonna 'ip.proto' in intero
df_clean_30m['ip.proto'] = df_clean_30m['ip.proto'].astype(int)
df_clean_30m['ip.checksum'] = df_clean_30m['ip.checksum'].apply(
    lambda x: int(x, 16) if pd.notnull(x) and isinstance(x, str) and len(x.strip()) > 0 else np.nan
)
df_clean_30m.loc[df_clean_30m['tcp.srcport'] == '0']
df_clean_30m['tcp.srcport'].replace({np.nan: '0'}, inplace=True)
df_clean_30m['tcp.dstport'].replace({np.nan: '0'}, inplace=True)
df_clean_30m['tcp.srcport'] = df_clean_30m['tcp.srcport'].apply(lambda x : int(x))
df_clean_30m['tcp.dstport'] = df_clean_30m['tcp.dstport'].apply(lambda x : int(x))
df_clean_30m['tcp.stream'] = df_clean_30m['tcp.stream'].astype(float)
df_clean_30m['tcp.len'] = df_clean_30m['tcp.len'].astype(float)
df_clean_30m['tcp.seq'] = df_clean_30m['tcp.seq'].astype(float)
df_clean_30m['tcp.nxtseq'] = df_clean_30m['tcp.nxtseq'].astype(float)
df_clean_30m['tcp.ack'] = df_clean_30m['tcp.ack'].astype(float)
df_clean_30m['tcp.hdr_len'] = df_clean_30m['tcp.hdr_len'].astype(float)
df_clean_30m['tcp.window_size_value'] = df_clean_30m['tcp.window_size_value'].astype(float)
df_clean_30m['tcp.window_size'] = df_clean_30m['tcp.window_size'].astype(float)
df_clean_30m['tcp.window_size_scalefactor'] = df_clean_30m['tcp.window_size_scalefactor'].astype(float)
df_clean_30m['tcp.urgent_pointer'] = df_clean_30m['tcp.urgent_pointer'].astype(float)
df_clean_30m['tcp.flags'] = df_clean_30m['tcp.flags'].apply(
    lambda x: int(x, 16) if pd.notnull(x) and isinstance(x, str) and len(x.strip()) > 0 else np.nan
)

# Stesso approccio per 'tcp.checksum'
df_clean_30m['tcp.checksum'] = df_clean_30m['tcp.checksum'].apply(
    lambda x: int(x, 16) if pd.notnull(x) and isinstance(x, str) and len(x.strip()) > 0 else np.nan
)


df_clean_30m['mbtcp.trans_id'] = df_clean_30m['mbtcp.trans_id'].astype(float)
df_clean_30m['mbtcp.prot_id'] = df_clean_30m['mbtcp.prot_id'].astype(float)
df_clean_30m['mbtcp.len'] = df_clean_30m['mbtcp.len'].astype(float)
df_clean_30m['mbtcp.unit_id'] = df_clean_30m['mbtcp.unit_id'].astype(float)







C:\Users\User\AppData\Local\Temp\ipykernel_12680\4189383700.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean_30m['ip.src'].replace({np.nan: '169.254.0.0'}, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12680\4189383700.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave